In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy


# Import Required Libraries & Data Loading

In [2]:
#importing the training data
df=pd.read_csv('IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [3]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [6]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
spacy.load('en_core_web_sm')

In [7]:
# general Settings

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 50
NUM_EPOCHS = 20
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 400
HIDDEN_DIM = 256
NUM_CLASSES = 2

# Text & label Preparation

In [8]:
!pip install -U torchtext==0.9.0

In [9]:
# Define feature processing
"""
Define the fields for the data.
"""
import torch
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [10]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [11]:
"""
Define the fields for the data.
"""

df.to_csv('moviedata.csv', index = None)
df = pd.read_csv('moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
# process the dataset

fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
                    path = '/content/moviedata.csv', 
                    format = 'csv', 
                    skip_header = True, 
                    fields =  fields 
)

# Data Split

In [13]:
# Split dataset into train and test set

train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [14]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization

In [15]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Flipping', 'through', 'the', 'channels', 'I', 'was', 'lucky', 'enough', 'to', 'stumble', 'upon', 'the', 'beginning', 'of', 'this', 'movie', '.', 'I', 'must', 'admit', 'that', 'it', 'grabbed', 'my', 'attention', 'almost', 'immediately', '.', 'I', 'love', 'older', 'films', 'and', 'this', 'is', 'or', 'should', 'be', 'considered', 'a', 'classic', '!', 'One', 'of', 'the', 'most', 'wonderful', 'rarities', 'of', 'this', 'movie', 'is', 'that', 'the', 'main', 'character', 'was', 'not', 'only', 'female', 'but', 'she', 'was', 'also', 'a', 'bad', 'girl', '.', 'I', 'highly', 'recommend', 'this', 'movie', '!'], 'LABEL_COLUMN_NAME': '1'}


In [16]:
# Build Vocabulary

TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 2


 2 extra value in vocabulary is because added (unknown) and (padding)

In [17]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
most_common_words = TEXT.vocab.freqs.most_common(20)
print(most_common_words)

[('the', 390972), (',', 369444), ('.', 318719), ('a', 210502), ('and', 210006), ('of', 194658), ('to', 180163), ('is', 145895), ('in', 118266), ('I', 105681), ('it', 103588), ('that', 93995), ('"', 85530), ("'s", 83147), ('this', 81771), ('-', 70411), ('/><br', 68787), ('was', 67372), ('as', 57734), ('movie', 57571)]


In [18]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


# Data Preparation for Batch wise Implimentation

In [19]:
# Define Dataloader

train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data),
        batch_size = 50,
        sort_within_batch = False,
        sort_key = lambda x:len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )

In [20]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([930, 50])
Target vector size: torch.Size([50])

Valid:
Text matrix size: torch.Size([47, 50])
Target vector size: torch.Size([50])

Test:
Text matrix size: torch.Size([41, 50])
Target vector size: torch.Size([50])


# Model Building

In [21]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.LSTM(embedding_dim, hidden_dim)        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        
        embedded = self.embedding(text)
        output,(hidden,cell) = self.rnn(embedded)
        hidden.squeeze_(0)
        output = self.fc(hidden)
        
        ### END YOUR CODE ###
        return output

In [22]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim= len(TEXT.vocab), ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim=EMBEDDING_DIM, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim=HIDDEN_DIM, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim=2  ### ADD NUMBER OF CLASSES HERE ###
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr= LEARNING_RATE)

# Define Accuracy

In [23]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [24]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        pred = model(text)
        loss = torch.nn.functional.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/020 | Batch 000/680 | Loss: 0.6976
Epoch: 001/020 | Batch 050/680 | Loss: 0.6976
Epoch: 001/020 | Batch 100/680 | Loss: 0.6899
Epoch: 001/020 | Batch 150/680 | Loss: 0.6944
Epoch: 001/020 | Batch 200/680 | Loss: 0.6849
Epoch: 001/020 | Batch 250/680 | Loss: 0.6940
Epoch: 001/020 | Batch 300/680 | Loss: 0.7014
Epoch: 001/020 | Batch 350/680 | Loss: 0.6941
Epoch: 001/020 | Batch 400/680 | Loss: 0.6894
Epoch: 001/020 | Batch 450/680 | Loss: 0.6952
Epoch: 001/020 | Batch 500/680 | Loss: 0.7027
Epoch: 001/020 | Batch 550/680 | Loss: 0.7238
Epoch: 001/020 | Batch 600/680 | Loss: 0.6923
Epoch: 001/020 | Batch 650/680 | Loss: 0.6883
training accuracy: 50.39%
valid accuracy: 49.92%
Time elapsed: 1.48 min
Epoch: 002/020 | Batch 000/680 | Loss: 0.6871
Epoch: 002/020 | Batch 050/680 | Loss: 0.6908
Epoch: 002/020 | Batch 100/680 | Loss: 0.6900
Epoch: 002/020 | Batch 150/680 | Loss: 0.6970
Epoch: 002/020 | Batch 200/680 | Loss: 0.6534
Epoch: 002/020 | Batch 250/680 | Loss: 0.6968
Epoch: 0

# Model Testing

In [25]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.45970046520233154

In [26]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability positive:


0.033389680087566376